In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets-classificationsynopsis/final_combined_movies_5genres.csv


In [2]:
# -*- coding: utf-8 -*-
"""
Movie Genre Classification with IndoBERT
Environment: Kaggle
"""

!pip install optuna


In [3]:
# BAGIAN PERTAMA - Import dan Konfigurasi
import os
import logging
import datetime
import json
import argparse
import gc
import sys
import codecs
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union

import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import psutil
import optuna
from optuna.trial import Trial

# Define Base Path for Kaggle
BASE_PATH = Path('/kaggle/working/genrematics-optuna-app')

In [4]:
# Configuration Constants
class Config:
    # Model Parameters dengan fixed values
    MODEL_PARAMS = {
        'EPOCHS': 20,
        'BATCH_SIZE': 16,  # Fixed
        'LEARNING_RATE': 2e-5,  # Fixed
        'MAX_LENGTH': 512,
        'TEST_SIZE': 0.15,
        'WEIGHT_DECAY': 0.01,  # Fixed
        'MIXUP_PROB': 0.3,  # Fixed
        'PATIENCE': 5,
        'SMOOTHING': 0.1  # Fixed
    }

    # Optimization Parameters dengan fixed options
    OPTIM_PARAMS = {
        'batch_size': [8, 16, 32],  # Fixed values
        'learning_rate': [1e-5, 2e-5, 3e-5],
        'weight_decay': [0.01, 0.02],
        'mixup_prob': [0.2, 0.3],
        'smoothing': [0.1, 0.15]
    }

    # Paths Configuration untuk Kaggle
    BASE_DIR = BASE_PATH
    DATA_DIR = Path('/kaggle/input/datasets-classificationsynopsis')
    LOG_DIR = BASE_DIR / 'logs'
    BACKUP_DIR = BASE_DIR / 'backups'
    TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    EXPERIMENT_DIR = LOG_DIR / 'experiments' / TIMESTAMP

    # Model and Data Paths
    MODEL_SAVE_DIR = EXPERIMENT_DIR / 'model'
    TOKENIZER_SAVE_DIR = EXPERIMENT_DIR / 'tokenizer'
    METRICS_DIR = EXPERIMENT_DIR / 'metrics'
    PLOTS_DIR = EXPERIMENT_DIR / 'plots'
    CM_DIR = PLOTS_DIR / 'confusion_matrices'

    # Model Files
    MODEL_BEST_ACC = MODEL_SAVE_DIR / "best_accuracy"
    MODEL_BEST_LOSS = MODEL_SAVE_DIR / "best_loss"
    TOKENIZER_BEST_ACC = TOKENIZER_SAVE_DIR / "best_accuracy"
    TOKENIZER_BEST_LOSS = TOKENIZER_SAVE_DIR / "best_loss"
    DATA_PATH = DATA_DIR / "final_combined_movies_5genres.csv"

    # Device Configuration
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    SAMPLE_SIZE: Optional[int] = None

    @classmethod
    def create_directories(cls) -> None:
        """Create all necessary directories in Kaggle working directory"""
        directories = [
            cls.LOG_DIR, cls.BACKUP_DIR,
            cls.EXPERIMENT_DIR, cls.MODEL_SAVE_DIR, cls.TOKENIZER_SAVE_DIR,
            cls.METRICS_DIR, cls.PLOTS_DIR, cls.CM_DIR
        ]
        for dir_path in directories:
            dir_path.mkdir(parents=True, exist_ok=True)
            print(f"Created directory: {dir_path}")

    @classmethod
    def setup_logging(cls) -> None:
        """Setup logging configuration untuk Kaggle"""
        log_file = cls.EXPERIMENT_DIR / 'training.log'
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file, encoding='utf-8', mode='a'),
                logging.StreamHandler(sys.stdout)
            ]
        )
        logging.info(f"Log file created at: {log_file}")

# Environment Check Function
def check_environment() -> bool:
    """Verify Kaggle environment and paths"""
    try:
        # Check if data directory exists
        if not Config.DATA_DIR.exists():
            raise RuntimeError(f"Data directory tidak ditemukan di: {Config.DATA_DIR}")

        # Check if dataset exists
        if not Config.DATA_PATH.exists():
            raise RuntimeError(f"Dataset tidak ditemukan di: {Config.DATA_PATH}")

        # Check GPU availability
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert to GB
            print(f"GPU tersedia: {gpu_name}")
            print(f"GPU Memory: {gpu_memory:.2f} GB")
        else:
            print("WARNING: GPU tidak tersedia, menggunakan CPU")

        return True
    except Exception as e:
        print(f"Error dalam setup environment: {str(e)}")
        return False

# Memory Management
class ModelManager:
    """Context manager for model memory management"""
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __enter__(self):
        return self.model, self.tokenizer

    def __exit__(self, exc_type, exc_val, exc_tb):
        del self.model
        del self.tokenizer
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

# Error Handling
def error_handler(func):
    """Decorator for consistent error handling"""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Error in {func.__name__}: {str(e)}")
            raise
    return wrapper

# Utility Functions
def get_memory_usage() -> float:
    """Get current memory usage of the program"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # in MB

def log_memory(step_name: str) -> None:
    """Log memory usage with consistent format"""
    memory = get_memory_usage()
    logging.info(f"Memory usage after {step_name}: {memory:.2f} MB")

def log_system_info() -> None:
    """Log system information including GPU details"""
    logging.info("System Information:")
    logging.info(f"Python Version: {sys.version}")
    logging.info(f"CPU Count: {os.cpu_count()}")
    logging.info(f"Initial Memory Usage: {get_memory_usage():.2f} MB")
    if torch.cuda.is_available():
        logging.info(f"GPU Device: {torch.cuda.get_device_name(0)}")
        logging.info(f"GPU Memory Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        logging.info(f"CUDA Version: {torch.version.cuda}")

# Create directories and setup logging
if check_environment():
    Config.create_directories()
    Config.setup_logging()
    log_system_info()
else:
    print("Failed to initialize environment. Please check the setup.")
    sys.exit(1)

GPU tersedia: Tesla T4
GPU Memory: 15.83 GB
Created directory: /kaggle/working/genrematics-optuna-app/logs
Created directory: /kaggle/working/genrematics-optuna-app/backups
Created directory: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331
Created directory: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/model
Created directory: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/tokenizer
Created directory: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/metrics
Created directory: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots
Created directory: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots/confusion_matrices
2025-02-10 22:03:31,806 - INFO - Log file created at: /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/training.log
2025-02-10 22:03:31,807 - INFO - System Information:
2025-02-10 22:03:31,807 - INFO - P

In [5]:
class MovieDataset(Dataset):
    """Dataset class untuk movie genre classification"""
    def __init__(self, texts: Union[List, np.ndarray],
                 labels: Union[List, np.ndarray],
                 tokenizer,
                 max_length: int = 512):
        # Input validation
        if not isinstance(texts, (list, np.ndarray)):
            raise ValueError("texts must be a list or numpy array")
        if not isinstance(labels, (list, np.ndarray)):
            raise ValueError("labels must be a list or numpy array")
        if len(texts) != len(labels):
            raise ValueError("texts and labels must have the same length")

        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        return len(self.texts)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Pastikan labels dalam format yang benar
        if isinstance(self.labels[idx], (list, np.ndarray)):
            labels = self.labels[idx]
        else:
            labels = [self.labels[idx]]

        return {
            'input_ids': encoding['input_ids'].flatten().long(),
            'attention_mask': encoding['attention_mask'].flatten().long(),
            'labels': torch.FloatTensor(labels)
        }

class DataProcessor:
    """Class for handling data preprocessing and loading"""
    @staticmethod
    def clean_text(text: str) -> str:
        """Clean and preprocess text data"""
        if isinstance(text, str):
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
            text = re.sub(r'\S+@\S+', '', text)
            text = re.sub(r'[^\w\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            return text.strip().lower()
        return ''

    @staticmethod
    @error_handler
    def load_and_preprocess_data(data_path: Path, sample_size: Optional[int] = None) -> pd.DataFrame:
        """Load and preprocess data with proper encoding handling"""
        if not data_path.exists():
            raise FileNotFoundError(f"Data file not found: {data_path}")

        if sample_size is not None and (not isinstance(sample_size, int) or sample_size <= 0):
            raise ValueError("sample_size must be a positive integer")

        logging.info("Loading and preprocessing data...")
        log_memory("start")
        initial_size = None

        # Try different encodings for Kaggle compatibility
        encodings_to_try = ['utf-8', 'utf-8-sig', 'latin1', 'iso-8859-1', 'cp1252']
        df = None

        for encoding in encodings_to_try:
            try:
                df = pd.read_csv(data_path, encoding=encoding)
                logging.info(f"Successfully loaded data using {encoding} encoding")
                initial_size = len(df)
                break
            except (UnicodeDecodeError, UnicodeError):
                continue

        if df is None:
            raise UnicodeError(f"Failed to read file with any of these encodings: {encodings_to_try}")

        log_memory("data loading")

        # Sample data if requested
        if sample_size:
            if sample_size > initial_size:
                logging.warning(f"Requested sample_size ({sample_size}) is larger than dataset size ({initial_size})")
                sample_size = initial_size
            logging.info(f"Taking sample of {sample_size} from {initial_size} total samples")
            df = df.head(sample_size)
        else:
            logging.info(f"Using full dataset with {initial_size} samples")

        # Log sample data
        logging.info("\nSample data:")
        for i, row in df.head(3).iterrows():
            logging.info(f"\nSample {i+1}:")
            logging.info(f"Synopsis: {row['sinopsis'][:100]}...")
            logging.info(f"Genre: {row['genre']}")

        # Preprocess data
        logging.info("\nPreprocessing text data...")
        tqdm.pandas()
        df['sinopsis'] = df['sinopsis'].progress_apply(DataProcessor.clean_text)
        df['genre'] = df['genre'].str.split(',')
        df = df.dropna(subset=['sinopsis', 'genre'])

        # Calculate and log dataset statistics
        stats = DataProcessor.calculate_dataset_statistics(df)

        # Save statistics to file
        stats_file = Config.METRICS_DIR / 'dataset_statistics.json'
        with open(stats_file, 'w', encoding='utf-8') as f:
            json.dump(stats, f, indent=4, ensure_ascii=False)

        log_memory("preprocessing")
        return df

    @staticmethod
    def prepare_data(df: pd.DataFrame, mlb: MultiLabelBinarizer) -> Tuple:
        """Prepare data for training"""
        genre_labels = mlb.fit_transform(df['genre'])
        return train_test_split(
            df['sinopsis'].values,
            genre_labels,
            test_size=Config.MODEL_PARAMS['TEST_SIZE'],
            random_state=42,
            stratify=genre_labels if len(genre_labels.shape) == 1 else None
        )

    @staticmethod
    def create_weighted_sampler(genre_labels: np.ndarray) -> WeightedRandomSampler:
        """Create weighted sampler for balanced batch sampling"""
        logging.info("Creating weighted sampler for balanced batch sampling...")

        sample_weights = np.zeros(len(genre_labels))
        for i in range(genre_labels.shape[1]):
            sample_weights += genre_labels[:, i] * (1.0 / np.sum(genre_labels[:, i]))

        sample_weights = sample_weights / sample_weights.sum()
        sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )

        logging.info(f"Created sampler with {len(sample_weights)} weights")
        return sampler

    @staticmethod
    def calculate_class_weights(genre_labels: np.ndarray, mlb: MultiLabelBinarizer) -> torch.Tensor:
        """Calculate class weights for handling imbalanced data"""
        class_weights = []
        logging.info("\nCalculating class weights for handling imbalanced data...")

        for i in range(genre_labels.shape[1]):
            genre = mlb.classes_[i]
            positive_samples = np.sum(genre_labels[:, i])
            total_samples = len(genre_labels)

            weights = compute_class_weight(
                class_weight='balanced',
                classes=np.array([0, 1]),
                y=genre_labels[:, i]
            )
            class_weights.append(weights[1])

            logging.info(f"{genre}:")
            logging.info(f"  Positive samples: {positive_samples}")
            logging.info(f"  Negative samples: {total_samples - positive_samples}")
            logging.info(f"  Weight: {weights[1]:.2f}")

        return torch.FloatTensor(class_weights).to(Config.DEVICE)

    @staticmethod
    def calculate_dataset_statistics(df: pd.DataFrame) -> Dict:
        """Calculate comprehensive dataset statistics"""
        logging.info("Calculating dataset statistics...")
        stats = {
            'general_stats': {
                'total_samples': len(df),
                'unique_genres': len(set([g for genres in df['genre'] for g in genres])),
                'avg_synopsis_length': float(df['sinopsis'].str.len().mean()),
                'null_values': df.isnull().sum().to_dict()
            },
            'genre_stats': DataProcessor.analyze_genre_combinations(df['genre'].values),
            'text_stats': DataProcessor.calculate_text_statistics(df['sinopsis'].values)
        }

        logging.info("\nDataset Statistics:")
        logging.info(f"Total samples: {stats['general_stats']['total_samples']}")
        logging.info(f"Unique genres: {stats['general_stats']['unique_genres']}")

        return stats

    @staticmethod
    def calculate_text_statistics(texts: np.ndarray) -> Dict:
        """Calculate detailed text statistics"""
        tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
        token_lengths = []
        char_lengths = []
        truncated_count = 0

        logging.info("Calculating text statistics...")
        for text in tqdm(texts, desc="Analyzing texts"):
            tokens = tokenizer.tokenize(text)
            token_lengths.append(len(tokens))
            char_lengths.append(len(text))
            if len(tokens) > Config.MODEL_PARAMS['MAX_LENGTH']:
                truncated_count += 1

        return {
            'avg_token_length': float(np.mean(token_lengths)),
            'max_token_length': int(np.max(token_lengths)),
            'min_token_length': int(np.min(token_lengths)),
            'avg_char_length': float(np.mean(char_lengths)),
            'max_char_length': int(np.max(char_lengths)),
            'min_char_length': int(np.min(char_lengths)),
            'truncated_sequences': truncated_count,
            'total_sequences': len(texts),
            'truncation_percentage': float(truncated_count / len(texts) * 100)
        }

    @staticmethod
    def analyze_genre_combinations(genres: List[List[str]]) -> Dict:
        """Analyze genre combination patterns"""
        logging.info("Analyzing genre combinations...")
        combinations = []
        for genre_list in genres:
            combinations.append(','.join(sorted(genre_list)))

        combination_counts = pd.Series(combinations).value_counts()

        return {
            'total_combinations': len(combination_counts),
            'unique_combinations': combination_counts.to_dict(),
            'top_combinations': combination_counts.head(10).to_dict(),
            'single_genre_count': sum(len(g) == 1 for g in genres),
            'multi_genre_count': sum(len(g) > 1 for g in genres),
            'max_genres_per_item': max(len(g) for g in genres),
            'avg_genres_per_item': float(np.mean([len(g) for g in genres]))
        }

In [6]:
class ModelSetup:
    """Class for handling model setup and data loaders"""
    @staticmethod
    def setup_model_and_tokenizer(num_labels: int) -> Tuple:
        """Setup model dan tokenizer"""
        logging.info("Setting up model and tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
        model = AutoModelForSequenceClassification.from_pretrained(
            'indobenchmark/indobert-base-p1',
            num_labels=num_labels,
            problem_type="multi_label_classification"
        ).to(Config.DEVICE)
        logging.info("Model and tokenizer setup completed")
        return model, tokenizer

    @staticmethod
    def setup_dataloaders(X_train: np.ndarray,
                         X_test: np.ndarray,
                         y_train: np.ndarray,
                         y_test: np.ndarray,
                         tokenizer,
                         batch_size: int) -> Tuple:
        """Setup data loaders"""
        logging.info("Setting up data loaders...")
        train_dataset = MovieDataset(X_train, y_train, tokenizer)
        val_dataset = MovieDataset(X_test, y_test, tokenizer)
        sampler = DataProcessor.create_weighted_sampler(y_train)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=sampler,
            num_workers=2,  # Adjusted for Kaggle
            pin_memory=True if torch.cuda.is_available() else False
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            num_workers=2,  # Adjusted for Kaggle
            pin_memory=True if torch.cuda.is_available() else False
        )

        logging.info(f"Created data loaders with batch size {batch_size}")
        logging.info(f"Training batches: {len(train_loader)}")
        logging.info(f"Validation batches: {len(val_loader)}")
        
        return train_loader, val_loader

    @staticmethod
    def validate_model_setup(model, tokenizer, num_labels: int) -> bool:
        """Validate model and tokenizer setup"""
        try:
            # Check model configuration
            if model.num_labels != num_labels:
                raise ValueError(f"Model has {model.num_labels} labels, expected {num_labels}")
            
            # Validate tokenizer
            sample_text = "Contoh teks untuk validasi"
            tokens = tokenizer(
                sample_text,
                max_length=Config.MODEL_PARAMS['MAX_LENGTH'],
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            # Check token shapes
            if tokens['input_ids'].shape[1] > Config.MODEL_PARAMS['MAX_LENGTH']:
                raise ValueError("Tokenizer producing sequences longer than max_length")
            
            # Test model forward pass
            with torch.no_grad():
                outputs = model(**tokens)
                if outputs.logits.shape[1] != num_labels:
                    raise ValueError(f"Model output shape mismatch: got {outputs.logits.shape[1]}, expected {num_labels}")
            
            logging.info("Model and tokenizer validation successful")
            return True
            
        except Exception as e:
            logging.error(f"Model validation failed: {str(e)}")
            return False

    @staticmethod
    def log_model_info(model) -> None:
        """Log model architecture and parameters"""
        try:
            total_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            
            logging.info("\nModel Information:")
            logging.info(f"Model Type: {model.__class__.__name__}")
            logging.info(f"Total Parameters: {total_params:,}")
            logging.info(f"Trainable Parameters: {trainable_params:,}")
            logging.info(f"Non-trainable Parameters: {total_params - trainable_params:,}")
            
            # Log memory usage
            if torch.cuda.is_available():
                memory_allocated = torch.cuda.memory_allocated() / 1024**2
                memory_reserved = torch.cuda.memory_reserved() / 1024**2
                logging.info(f"GPU Memory Allocated: {memory_allocated:.2f} MB")
                logging.info(f"GPU Memory Reserved: {memory_reserved:.2f} MB")
                
        except Exception as e:
            logging.error(f"Error logging model info: {str(e)}")

In [7]:
class LossFunctions:
    """Class untuk menangani berbagai loss functions"""

    @staticmethod
    def focal_loss(predictions: torch.Tensor,
                  targets: torch.Tensor,
                  gamma: float = 2.0,
                  alpha: float = 0.25) -> torch.Tensor:
        """Calculate focal loss for multi-label classification"""
        ce_loss = F.binary_cross_entropy_with_logits(predictions, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = alpha * (1-pt)**gamma * ce_loss
        return focal_loss.mean()

    @staticmethod
    def label_smoothing_loss(outputs: torch.Tensor,
                           targets: torch.Tensor,
                           smoothing: float) -> torch.Tensor:
        """Calculate loss with label smoothing"""
        log_probs = F.log_softmax(outputs, dim=-1)
        targets = torch.clamp(targets * (1.0 - smoothing), min=smoothing / (targets.size(-1) - 1))
        return torch.mean(torch.sum(-targets * log_probs, dim=-1))

class DataAugmentation:
    """Class untuk menangani augmentasi data"""

    @staticmethod
    def apply_mixup(batch: Dict[str, torch.Tensor], alpha: float = 0.2) -> Dict[str, torch.Tensor]:
        """Apply mixup augmentation to batch"""
        # Move tensors to device
        input_ids = batch['input_ids'].to(Config.DEVICE)
        attention_mask = batch['attention_mask'].to(Config.DEVICE)
        labels = batch['labels'].to(Config.DEVICE)

        lam = np.random.beta(alpha, alpha)
        mixed_input_ids = lam * input_ids + (1 - lam) * input_ids.flip(0)
        mixed_attention_mask = lam * attention_mask + (1 - lam) * attention_mask.flip(0)
        mixed_labels = lam * labels + (1 - lam) * labels.flip(0)

        return {
            'input_ids': mixed_input_ids.long(),
            'attention_mask': mixed_attention_mask.long(),
            'labels': mixed_labels
        }

class Visualization:
    """Class untuk menangani visualisasi data"""
    
    @staticmethod
    def plot_training_history(history_data: Dict) -> None:
        """Plot and save training metrics visualization with consistent dict structure
        
        Args:
            history_data: Dictionary containing training history with keys:
                - epoch: List of epoch numbers
                - train_loss: List of training losses
                - val_loss: List of validation losses 
                - val_f1: List of validation F1 scores
                - val_precision: List of validation precision scores
                - val_recall: List of validation recall scores
        """
        plt.figure(figsize=(20, 12))

        # Loss Plot
        plt.subplot(2, 2, 1)
        plt.plot(history_data['epoch'], history_data['train_loss'],
                label='Training Loss', marker='o', linewidth=2)
        plt.plot(history_data['epoch'], history_data['val_loss'],
                label='Validation Loss', marker='o', linewidth=2)
        plt.title('Training vs Validation Loss', fontsize=12, pad=15)
        plt.xlabel('Epoch', fontsize=10)
        plt.ylabel('Loss', fontsize=10)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)

        # F1 Score Plot 
        plt.subplot(2, 2, 2)
        plt.plot(history_data['epoch'], history_data['val_f1'],
                label='F1 Score', marker='o', color='green', linewidth=2)
        plt.title('F1 Score Over Time', fontsize=12, pad=15)
        plt.xlabel('Epoch', fontsize=10)
        plt.ylabel('F1 Score', fontsize=10)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)

        # Learning Curve
        plt.subplot(2, 2, 3)
        loss_diff = np.array(history_data['train_loss']) - np.array(history_data['val_loss'])
        plt.plot(history_data['epoch'], loss_diff,
                label='Train-Val Loss Difference', marker='o', color='purple', linewidth=2)
        plt.axhline(y=0, color='r', linestyle='--', label='Ideal Difference')
        plt.title('Learning Curve', fontsize=12, pad=15)
        plt.xlabel('Epoch', fontsize=10)
        plt.ylabel('Loss Difference', fontsize=10)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)

        # Save plot
        plot_path = Config.PLOTS_DIR / 'training_history.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close()

        logging.info(f"Training history plots saved to {plot_path}")

    @staticmethod
    def plot_confusion_matrices(predictions: np.ndarray,
                              true_labels: np.ndarray,
                              class_names: List[str]) -> None:
        """Plot confusion matrices for each class
        
        Args:
            predictions: Binary predictions array
            true_labels: True binary labels array
            class_names: List of class names
        """
        for idx, class_name in enumerate(class_names):
            cm = confusion_matrix(true_labels[:, idx], predictions[:, idx])
            
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title(f'Confusion Matrix - {class_name}')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            
            plot_path = Config.CM_DIR / f'confusion_matrix_{class_name}.png'
            plt.savefig(plot_path, dpi=300, bbox_inches='tight')
            plt.close()

        logging.info(f"Training history plots saved to {plot_path}")

In [8]:
class ModelEvaluator:
    """Class untuk evaluasi model"""

    @staticmethod
    @error_handler
    def evaluate_model(model: torch.nn.Module,
                      val_loader: DataLoader,
                      mlb: MultiLabelBinarizer) -> Dict:
        """Evaluate model performance"""
        model.eval()
        all_preds = []
        all_labels = []
        total_loss = 0
        n_batches = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Evaluating"):
                input_ids = batch['input_ids'].to(Config.DEVICE)
                attention_mask = batch['attention_mask'].to(Config.DEVICE)
                labels = batch['labels'].to(Config.DEVICE)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = LossFunctions.focal_loss(outputs.logits, labels)
                total_loss += loss.item()
                n_batches += 1

                preds = torch.sigmoid(outputs.logits).cpu().numpy()
                preds_binary = (preds > 0.5).astype(int)
                
                all_preds.extend(preds_binary)
                all_labels.extend(labels.cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        metrics = {
            'loss': total_loss / n_batches,
            'macro_f1': f1_score(all_labels, all_preds, average='macro'),
            'macro_precision': precision_score(all_labels, all_preds, average='macro'),
            'macro_recall': recall_score(all_labels, all_preds, average='macro'),
            'per_class': {
                genre: {
                    'f1': f1_score(all_labels[:, i], all_preds[:, i]),
                    'precision': precision_score(all_labels[:, i], all_preds[:, i]),
                    'recall': recall_score(all_labels[:, i], all_preds[:, i])
                }
                for i, genre in enumerate(mlb.classes_)
            }
        }

        # Plot confusion matrices
        Visualization.plot_confusion_matrices(all_preds, all_labels, mlb.classes_)

        return metrics

class ModelTrainer:
    """Class untuk menangani training model"""
    
    @staticmethod
    def train(model: torch.nn.Module,
             train_loader: DataLoader,
             val_loader: DataLoader,
             mlb: MultiLabelBinarizer,
             n_epochs: int) -> Dict:
        """Full training loop dengan history tracking yang konsisten
        
        Args:
            model: Model PyTorch yang akan dilatih
            train_loader: DataLoader untuk data training
            val_loader: DataLoader untuk data validasi
            mlb: MultiLabelBinarizer yang sudah difit
            n_epochs: Jumlah epoch untuk training
            
        Returns:
            Dict berisi history training dan metrics terbaik
        """
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=Config.MODEL_PARAMS['LEARNING_RATE'],
            weight_decay=Config.MODEL_PARAMS['WEIGHT_DECAY']
        )
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=len(train_loader) * n_epochs
        )

        best_metrics = None
        best_loss = float('inf')
        patience_counter = 0
        
        history = {
            'epoch': [],
            'train_loss': [],
            'val_loss': [],
            'val_f1': [],
            'val_precision': [],
            'val_recall': []
        }

        start_time = datetime.datetime.now()

        for epoch in range(n_epochs):
            epoch_start_time = datetime.datetime.now()
            
            # Training phase
            train_metrics = ModelTrainer.train_epoch(
                model=model,
                train_loader=train_loader,
                optimizer=optimizer,
                scheduler=scheduler,
                epoch=epoch + 1
            )
            
            # Validation phase
            val_metrics = ModelEvaluator.evaluate_model(model, val_loader, mlb)
            
            # Update training history
            history['epoch'].append(epoch + 1)
            history['train_loss'].append(train_metrics['loss'])
            history['val_loss'].append(val_metrics['loss'])
            history['val_f1'].append(val_metrics['macro_f1'])
            history['val_precision'].append(val_metrics['macro_precision'])
            history['val_recall'].append(val_metrics['macro_recall'])

            # Save best model based on F1 score
            if best_metrics is None or val_metrics['macro_f1'] > best_metrics['macro_f1']:
                best_metrics = val_metrics
                ModelTrainer.save_model(model, Config.MODEL_BEST_ACC)
                patience_counter = 0
            else:
                patience_counter += 1

            # Save best model based on loss
            if val_metrics['loss'] < best_loss:
                best_loss = val_metrics['loss']
                ModelTrainer.save_model(model, Config.MODEL_BEST_LOSS)

            # Early stopping check
            if patience_counter >= Config.MODEL_PARAMS['PATIENCE']:
                logging.info(f"Early stopping triggered after {epoch + 1} epochs")
                break

            # Calculate epoch time
            epoch_time = datetime.datetime.now() - epoch_start_time
            
            # Log metrics
            logging.info(
                f"Epoch {epoch+1}/{n_epochs} - Time: {epoch_time.total_seconds():.2f}s - "
                f"Train Loss: {train_metrics['loss']:.4f}, "
                f"Val Loss: {val_metrics['loss']:.4f}, "
                f"Val F1: {val_metrics['macro_f1']:.4f}, "
                f"Val Precision: {val_metrics['macro_precision']:.4f}, "
                f"Val Recall: {val_metrics['macro_recall']:.4f}"
            )

            # Plot training history
            Visualization.plot_training_history(history)
            
            # Save metrics to JSON for web app
            metrics_file = Config.METRICS_DIR / 'training_history.json'
            with open(metrics_file, 'w', encoding='utf-8') as f:
                json.dump({
                    'training_history': {
                        'epochs': history['epoch'],
                        'training_loss': history['train_loss'],
                        'validation_loss': history['val_loss'],
                        'accuracy': history['val_f1'],
                        'best_accuracy': max(history['val_f1']),
                        'best_val_loss': min(history['val_loss'])
                    },
                    'model_info': {
                        'classes': mlb.classes_.tolist(),
                        'total_samples': len(train_loader.dataset),
                        'training_time': str(datetime.datetime.now() - start_time),
                        'parameters': {
                            'total_parameters': sum(p.numel() for p in model.parameters()),
                            'trainable_parameters': sum(p.numel() for p in model.parameters() if p.requires_grad)
                        }
                    }
                }, f, indent=4)

            # Memory cleanup after each epoch
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

        total_time = datetime.datetime.now() - start_time
        logging.info(f"Training completed in {total_time}")
        
        return history, best_metrics

    @staticmethod
    def train_epoch(model: torch.nn.Module,
                   train_loader: DataLoader,
                   optimizer: torch.optim.Optimizer,
                   scheduler: torch.optim.lr_scheduler._LRScheduler,
                   epoch: int) -> Dict[str, float]:
        """Training untuk satu epoch
        
        Args:
            model: Model PyTorch yang akan dilatih
            train_loader: DataLoader untuk data training
            optimizer: Optimizer yang digunakan
            scheduler: Learning rate scheduler
            epoch: Nomor epoch saat ini
            
        Returns:
            Dict berisi metrics training untuk epoch ini
        """
        model.train()
        total_loss = 0
        n_batches = 0
        
        # Progress bar
        progress_bar = tqdm(train_loader, desc=f'Training Epoch {epoch}')
        
        for batch in progress_bar:
            # Move batch to device
            input_ids = batch['input_ids'].to(Config.DEVICE)
            attention_mask = batch['attention_mask'].to(Config.DEVICE)
            labels = batch['labels'].to(Config.DEVICE)
            
            # Clear gradients
            optimizer.zero_grad()
            
            # Apply mixup augmentation dengan probability
            if np.random.random() < Config.MODEL_PARAMS['MIXUP_PROB']:
                batch = DataAugmentation.apply_mixup(batch)
                input_ids = batch['input_ids']
                attention_mask = batch['attention_mask']
                labels = batch['labels']
            
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Calculate loss dengan label smoothing
            loss = LossFunctions.label_smoothing_loss(
                outputs.logits,
                labels,
                Config.MODEL_PARAMS['SMOOTHING']
            )
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Update weights
            optimizer.step()
            
            # Update learning rate
            scheduler.step()
            
            # Update metrics
            total_loss += loss.item()
            n_batches += 1
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'avg_loss': f'{total_loss/n_batches:.4f}',
                'lr': f'{scheduler.get_last_lr()[0]:.2e}'
            })
            
            # Memory cleanup every few batches
            if n_batches % 10 == 0:
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                gc.collect()
        
        # Calculate average loss
        avg_loss = total_loss / n_batches
        
        return {
            'loss': avg_loss,
            'learning_rate': scheduler.get_last_lr()[0]
        }

    @staticmethod
    def save_model(model: torch.nn.Module, path: Path) -> None:
        """Save model checkpoint dengan proper error handling
        
        Args:
            model: Model PyTorch yang akan disimpan
            path: Path untuk menyimpan model
        """
        try:
            # Create directory if not exists
            path.parent.mkdir(parents=True, exist_ok=True)
            
            # Prepare checkpoint data
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'config': model.config.to_dict(),
                'save_time': datetime.datetime.now().strftime("%Y%m%d_%H%M%S"),
                'device': str(next(model.parameters()).device)
            }
            
            # Save temporary file first
            temp_path = path.parent / f"{path.name}.tmp"
            torch.save(checkpoint, temp_path)
            
            # Atomic rename to final path
            temp_path.replace(path)
            
            logging.info(f"Model saved successfully to {path}")
            
        except Exception as e:
            logging.error(f"Error saving model: {str(e)}")
            if temp_path.exists():
                temp_path.unlink()
            raise

In [9]:
class HyperparameterOptimizer:
    @staticmethod
    def objective(trial: Trial, df: pd.DataFrame, mlb: MultiLabelBinarizer) -> float:
        try:
            params = HyperparameterOptimizer._get_trial_parameters(trial)
            logging.info(f"\nTrial {trial.number} Parameters:")
            for name, value in params.items():
                logging.info(f"{name}: {value}")

            X_train, X_test, y_train, y_test = DataProcessor.prepare_data(df, mlb)

            with ModelManager(*ModelSetup.setup_model_and_tokenizer(len(mlb.classes_))) as (model, tokenizer):
                train_loader, val_loader = ModelSetup.setup_dataloaders(
                    X_train, X_test, y_train, y_test,
                    tokenizer, params['batch_size']
                )

                optimizer = torch.optim.AdamW(
                    model.parameters(),
                    lr=params['learning_rate'],
                    weight_decay=params['weight_decay']
                )

                # Training loop untuk optimization
                best_f1 = 0.0
                for epoch in range(3):  # Reduced epochs for optimization
                    model.train()
                    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/3"):
                        optimizer.zero_grad()

                        if np.random.random() < params['mixup_prob']:
                            batch = DataAugmentation.apply_mixup(batch)

                        input_ids = batch['input_ids'].to(Config.DEVICE)
                        attention_mask = batch['attention_mask'].to(Config.DEVICE)
                        labels = batch['labels'].to(Config.DEVICE)

                        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                        loss = LossFunctions.label_smoothing_loss(
                            outputs.logits, labels, params['smoothing']
                        )

                        loss.backward()
                        optimizer.step()

                    # Evaluation
                    metrics = ModelEvaluator.evaluate_model(model, val_loader, mlb)
                    current_f1 = metrics['macro_f1']
                    best_f1 = max(best_f1, current_f1)

                    trial.report(current_f1, epoch)
                    if trial.should_prune():
                        raise optuna.TrialPruned()

                return best_f1

        except optuna.TrialPruned:
            raise
        except Exception as e:
            logging.error(f"Error in trial: {str(e)}")
            return float('-inf')

    @staticmethod
    def _get_trial_parameters(trial: Trial) -> Dict:
        params = {}
        try:
            for name, values in Config.OPTIM_PARAMS.items():
                params[name] = trial.suggest_categorical(name, values)
        except Exception as e:
            logging.error(f"Error getting trial parameters: {str(e)}")
            raise
        return params

    @staticmethod
    def optimize(df: pd.DataFrame, mlb: MultiLabelBinarizer, n_trials: int = 20) -> Dict:
        study = optuna.create_study(
            direction="maximize",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner()
        )

        study.optimize(
            lambda trial: HyperparameterOptimizer.objective(trial, df, mlb),
            n_trials=n_trials
        )

        logging.info("\nOptimization Results:")
        logging.info(f"Best trial:")
        logging.info(f"  Value: {study.best_trial.value:.4f}")
        logging.info("  Params:")
        for key, value in study.best_trial.params.items():
            logging.info(f"    {key}: {value}")

        results_file = Config.METRICS_DIR / 'optuna_results.json'
        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump({
                'best_trial': {
                    'number': study.best_trial.number,
                    'value': study.best_trial.value,
                    'params': study.best_trial.params
                },
                'all_trials': [{
                    'number': trial.number,
                    'value': trial.value,
                    'params': trial.params
                } for trial in study.trials if trial.value is not None]
            }, f, indent=4)

        return study.best_trial.params

In [10]:
class Args:
    def __init__(self):
        self.sample_size = None
        self.epochs = Config.MODEL_PARAMS['EPOCHS']
        self.batch_size = Config.MODEL_PARAMS['BATCH_SIZE']
        self.learning_rate = Config.MODEL_PARAMS['LEARNING_RATE']
        self.max_length = Config.MODEL_PARAMS['MAX_LENGTH']
        self.test_size = Config.MODEL_PARAMS['TEST_SIZE']
        self.weight_decay = Config.MODEL_PARAMS['WEIGHT_DECAY']
        self.mixup_prob = Config.MODEL_PARAMS['MIXUP_PROB']
        self.patience = Config.MODEL_PARAMS['PATIENCE']
        self.smoothing = Config.MODEL_PARAMS['SMOOTHING']
        self.n_trials = 20
        self.no_cuda = False
        self.seed = 42

def main():
    args = Args()
    
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

    try:
        if not check_environment():
            raise RuntimeError("Environment check failed!")

        logging.info("Starting movie genre classification")
        logging.info(f"Using device: {Config.DEVICE}")

        df = DataProcessor.load_and_preprocess_data(Config.DATA_PATH, Config.SAMPLE_SIZE)
        mlb = MultiLabelBinarizer()

        # Hyperparameter optimization
        logging.info("\nStarting hyperparameter optimization...")
        best_params = HyperparameterOptimizer.optimize(df, mlb, args.n_trials)

        # Update config with best parameters
        for param, value in best_params.items():
            if param in Config.MODEL_PARAMS:
                Config.MODEL_PARAMS[param] = value
                logging.info(f"Updated {param}: {value}")

        # Final training with best parameters
        X_train, X_test, y_train, y_test = DataProcessor.prepare_data(df, mlb)
        
        with ModelManager(*ModelSetup.setup_model_and_tokenizer(len(mlb.classes_))) as (model, tokenizer):
            train_loader, val_loader = ModelSetup.setup_dataloaders(
                X_train, X_test, y_train, y_test,
                tokenizer, Config.MODEL_PARAMS['BATCH_SIZE']
            )

            history, best_metrics = ModelTrainer.train(
                model, train_loader, val_loader, mlb, Config.MODEL_PARAMS['EPOCHS']
            )

            # Save final results
            results = {
                'config': Config.MODEL_PARAMS,
                'best_metrics': best_metrics,
                'history': history,
                'classes': mlb.classes_.tolist()
            }

            with open(Config.METRICS_DIR / 'final_results.json', 'w') as f:
                json.dump(results, f, indent=4)

        logging.info("\nTraining completed successfully!")

    except Exception as e:
        logging.error(f"Training failed: {str(e)}")
        raise
    finally:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

if __name__ == "__main__":
    main()


GPU tersedia: Tesla T4
GPU Memory: 15.83 GB
2025-02-10 22:03:31,980 - INFO - Starting movie genre classification
2025-02-10 22:03:31,981 - INFO - Using device: cuda
2025-02-10 22:03:31,982 - INFO - Loading and preprocessing data...
2025-02-10 22:03:31,984 - INFO - Memory usage after start: 635.15 MB
2025-02-10 22:03:32,048 - INFO - Successfully loaded data using utf-8 encoding
2025-02-10 22:03:32,049 - INFO - Memory usage after data loading: 637.70 MB
2025-02-10 22:03:32,050 - INFO - Using full dataset with 1738 samples
2025-02-10 22:03:32,051 - INFO - 
Sample data:
2025-02-10 22:03:32,052 - INFO - 
Sample 1:
2025-02-10 22:03:32,057 - INFO - Synopsis: Setelah kematian yang tampak, Siena mampu melihat tanda-tanda bahwa orang-orang akan meninggal. Namu...
2025-02-10 22:03:32,057 - INFO - Genre: Horor
2025-02-10 22:03:32,058 - INFO - 
Sample 2:
2025-02-10 22:03:32,060 - INFO - Synopsis: Alfi (Al Ghazali) bertemu dengan Alana (Caitlin Halderman), seorang siswa baru di sekolahnya. Ternya...

100%|██████████| 1738/1738 [00:00<00:00, 14018.25it/s]

2025-02-10 22:03:32,205 - INFO - Calculating dataset statistics...
2025-02-10 22:03:32,208 - INFO - Analyzing genre combinations...


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...

2025-02-11 00:16:47,185 - INFO - Training history plots saved to /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots/confusion_matrices/confusion_matrix_Romantis.png
2025-02-11 00:16:47,186 - INFO - Epoch 7/20 - Time: 149.98s - Train Loss: 0.8976, Val Loss: 0.0705, Val F1: 0.5053, Val Precision: 0.4084, Val Recall: 0.7064
2025-02-11 00:16:48,742 - INFO - Training history plots saved to /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots/training_history.png


Evaluating: 100%|██████████| 17/17 [00:07<00:00,  2.26it/s]


2025-02-11 00:19:19,807 - INFO - Training history plots saved to /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots/confusion_matrices/confusion_matrix_Romantis.png
2025-02-11 00:19:19,809 - INFO - Epoch 8/20 - Time: 149.98s - Train Loss: 0.9759, Val Loss: 0.0714, Val F1: 0.5207, Val Precision: 0.4215, Val Recall: 0.7136
2025-02-11 00:19:21,361 - INFO - Training history plots saved to /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots/training_history.png


Evaluating: 100%|██████████| 17/17 [00:07<00:00,  2.27it/s]


2025-02-11 00:21:52,781 - INFO - Training history plots saved to /kaggle/working/genrematics-optuna-app/logs/experiments/20250210_220331/plots/confusion_matrices/confusion_matrix_Romantis.png
2025-02-11 00:21:52,783 - INFO - Early stopping triggered after 9 epochs
2025-02-11 00:21:52,784 - INFO - Training completed in 0:22:49.858228
2025-02-11 00:21:53,935 - INFO - 
Training completed successfully!
